In [ ]:
import sympy as sym
sym.init_printing()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import InterpolatedUnivariateSpline

plt.style.use('./my_style.py')
import matplotlib.ticker as mticker

from cycler import cycler
import matplotlib.cm     as cm
from multiprocess import Pool

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

def interp_df_fun(col, k_in, k_out, deg):
    return InterpolatedUnivariateSpline(k_in, col, k=deg)(k_out)

def interp_df(df, N_points=1000, deg=3, mode='lin'):
    k_min = df.index.min()
    k_max = df.index.max()
    if mode=='lin':
        k_out = np.linspace(k_min, k_max, N_points)
    elif mode=='log':
        k_out = np.logspace(np.log10(k_min), np.log10(k_max), N_points)
    df = df.apply(lambda col: interp_df_fun(col, df.index, k_out, deg)) 
    df.index = k_out
    return df

In [ ]:
from solver import KmouExpansionJordan

In [ ]:
beta_arr = np.linspace(0.01,0.2, 5)
K0_arr = np.logspace(-1,4, 6)

In [ ]:
beta_mesh, K0_mesh =  np.meshgrid(beta_arr, K0_arr)

In [ ]:
beta_mesh_flat = beta_mesh.flatten()
K0_mesh_flat = K0_mesh.flatten()

In [ ]:
def get_BL(vec, a_val=1, frame='Jordan'):
    beta, K0 = vec
    print(f'Working on beta = {beta}, K0 = {K0}, ')
    dum_kmou = KmouExpansionJordan(beta=beta, K0=K0, lamb_val=1.5)
    dum_kmou.tune_lambda(E_target=1, a_target=1, frame=frame, maxiter=15)
    dum_kmou.get_growth()
    dum_kmou.get_growth_LCDM()
    if frame=='Jordan':
        a_J = a_val
    elif frame=='Einstein':
        a_J = dum_kmou.get_a_Jor(a_val)
    
    B_L0 = (dum_kmou.D_kmou_J.sol(a_J)[1]/dum_kmou.D_LCDM.sol(a_val)[1])**2
    return B_L0

get_BL_Ein = lambda vec: get_BL(vec, frame='Einstein')
get_BL_Jor = lambda vec: get_BL(vec, frame='Jordan')

In [ ]:
with Pool(8) as p:
    BL_list = list(p.map(get_BL_Ein, zip(beta_mesh_flat,K0_mesh_flat), ))

In [ ]:
with Pool(8) as p:
    BL_list_Jor = list(p.map(get_BL_Jor, zip(beta_mesh_flat,K0_mesh_flat), ))

In [ ]:
BL_arr = np.array(BL_list).reshape(len(K0_arr), len(beta_arr))

In [ ]:
BL_arr_Jor = np.array(BL_list_Jor).reshape(len(K0_arr), len(beta_arr))

In [ ]:
df_BL =pd.DataFrame(BL_arr.T, index=np.round(beta_arr,4), columns=np.round(K0_arr,4))

In [ ]:
df_BL_Jor =pd.DataFrame(BL_arr_Jor.T, index=np.round(beta_arr,4), columns=np.round(K0_arr,4))

In [ ]:
df_BL_fine = (interp_df(df_BL, N_points=100).T).T
# df_BL_fine = interp_df(interp_df(df_BL, N_points=100).T, 5, mode='log').T

In [ ]:
df_BL_fine_Jor = (interp_df(df_BL_Jor, N_points=100).T).T
# df_BL_fine = interp_df(interp_df(df_BL, N_points=100).T, 5, mode='log').T

In [ ]:
df_BL_st = df_BL_fine.stack().reset_index()
df_BL_st.columns = ['beta', 'K0', 'B_L']

df_BL_st['log_K0'] = np.log10(df_BL_st.K0)

In [ ]:
df_BL_st_Jor = df_BL_fine_Jor.stack().reset_index()
df_BL_st_Jor.columns = ['beta', 'K0', 'B_L']

df_BL_st_Jor['log_K0'] = np.log10(df_BL_st_Jor.K0)

In [ ]:
import seaborn as sns

In [ ]:
g = sns.scatterplot(df_BL_st, y='B_L', x='beta', hue='K0', palette='colorblind', )
plt.hlines(1, 0.05,0.2, colors='k', linestyles='--')
g.set_xlabel(r'$\beta$');
g.set_ylabel(r'$B_{\rm L}(z=0)$');
plt.title('Einstein frame tuning');

plt.savefig('./Figures/EinsteinFrameTuning_BL.pdf', format='pdf')

In [ ]:
g = sns.scatterplot(df_BL_st_Jor, y='B_L', x='beta', hue='K0', palette='colorblind', )
plt.hlines(1, 0.05,0.2, colors='k', linestyles='--')
g.set_xlabel(r'$\beta$');
g.set_ylabel(r'$B_{\rm L}(z=0)$');
plt.title('Jordan frame tuning');

plt.savefig('./Figures/JordanFrameTuning_BL.pdf', format='pdf')

In [ ]:
InterpolatedUnivariateSpline(df_BL_fine_Jor[1], df_BL_fine_Jor.index)(df_BL.loc[0.2,1])

In [ ]:
norm = plt.Normalize(df_BL_st['log_K0'].min(), df_BL_st['log_K0'].max())
sm = plt.cm.ScalarMappable(cmap="RdBu", norm=norm)
sm.set_array([])

g = sns.scatterplot(df_BL_st, y='B_L', x='beta', hue='log_K0', palette='RdBu', )
g.set_xlabel(r'$\beta$');
g.set_ylabel(r'$B_{\rm L}(z=0)$');

# Remove the legend and add a colorbar
g.get_legend().remove()
g.figure.colorbar(sm, label='$\log_{10}(K_0)$');

In [ ]:
df_BL_fine[]

In [ ]:
g = sns.heatmap(df_BL)

# Test results

In [ ]:
test_exp = KmouExpansionJordan(lamb_val=10, beta=0.124, K0=1)

test_exp.tune_lambda(maxiter=10)
test_exp.get_growth()

test_exp.get_growth_LCDM()

In [ ]:
a_vals = np.linspace(1e-3,1, 1000)
plt.semilogx(a_vals, test_exp.D_kmou_J.sol(a_vals)[1]/test_exp.D_LCDM.sol(a_vals)[1])

In [ ]:
out_table = test_exp.eval()
plt.semilogx(out_table[0],out_table[3]/test_exp.E_LCDM_fun(out_table[0]))